## Deliverable 2. Create a Customer Travel Destinations Map.

In [85]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,77.00,94,20,5.75,few clouds
1,1,Atuona,PF,-9.8000,-139.0333,80.40,77,87,18.90,overcast clouds
2,2,Chokurdakh,RU,70.6333,147.9167,-13.92,96,98,5.84,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,79.54,70,9,13.76,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,86.00,79,75,18.41,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.3428,57.7575,77.00,94,20,5.75,few clouds
1,1,Atuona,PF,-9.8000,-139.0333,80.40,77,87,18.90,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,79.54,70,9,13.76,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,86.00,79,75,18.41,broken clouds
7,7,Port Blair,IN,11.6667,92.7500,80.46,79,10,4.61,clear sky


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,77.00,few clouds,-20.3428,57.7575,
1,Atuona,PF,80.40,overcast clouds,-9.8000,-139.0333,
3,Rikitea,PF,79.54,clear sky,-23.1203,-134.9692,
4,Avarua,CK,86.00,broken clouds,-21.2078,-159.7750,
7,Port Blair,IN,80.46,clear sky,11.6667,92.7500,
8,Hithadhoo,MV,82.72,overcast clouds,-0.6000,73.0833,
11,Hilo,US,84.20,scattered clouds,19.7297,-155.0900,
14,Vaini,TO,75.20,moderate rain,-21.2000,-175.2000,
15,Champerico,GT,87.80,scattered clouds,14.3000,-91.9167,
17,Boa Vista,BR,80.60,light rain,2.8197,-60.6733,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [79]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df_clean = hotel_df.replace(r'^\s*$', np.NaN , regex=True)
clean_data_df = hotel_df_clean.dropna()
clean_data_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,77.00,few clouds,-20.3428,57.7575,Casa Tia Villa
1,Atuona,PF,80.40,overcast clouds,-9.8000,-139.0333,Villa Enata
3,Rikitea,PF,79.54,clear sky,-23.1203,-134.9692,Pension Maro'i
4,Avarua,CK,86.00,broken clouds,-21.2078,-159.7750,Paradise Inn
7,Port Blair,IN,80.46,clear sky,11.6667,92.7500,Welcomhotel Bay Island Port Blair
8,Hithadhoo,MV,82.72,overcast clouds,-0.6000,73.0833,Scoop Guest House
11,Hilo,US,84.20,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Vaini,TO,75.20,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
15,Champerico,GT,87.80,scattered clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
17,Boa Vista,BR,80.60,light rain,2.8197,-60.6733,Aipana Plaza Hotel


In [91]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [80]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [93]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))